In [3]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
import wandb
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, MBartForConditionalGeneration
from lstm import BiLSTM
from distill import beam_search_decode
from config_distill_v3_2 import (
    TEACHER_MODEL_NAME,
    MAX_SOURCE_LEN,
    MAX_TARGET_LEN,
    EMBED_DIM,
    ENC_HIDDEN_DIM,
    DEC_HIDDEN_DIM,
    NUM_LAYERS,
    DROPOUT,
    BATCH_SIZE,
    LEARNING_RATE,
    NUM_EPOCHS,
    TEMPERATURE,
    BEAM_SIZE,
    BEAM_MAX_LENGTH,
    MIN_LEN,
)


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_text = """Приложение 1

УТВЕРЖДЕНО
приказом НИУ ВШЭ  Пермь
от 19.10.2023 8.2.6.2-10191023-2

ПОЛОЖЕНИЕ
о профильной олимпиаде НИУ ВШЭ  Пермь для школьников 10-11 классов

Общие положения
Положение о профильной олимпиаде НИУ ВШЭ  Пермь для школьников 10-11 классов (далее Положение) разработано на основании Федерального закона от 29.12.2012  273-ФЗ Об образовании в Российской Федерации (далее Закон  273-ФЗ) и определяет порядок проведения профильной олимпиады НИУ ВШЭ  Пермь для школьников 10-11 классов (далее Олимпиада), ее организационное и методическое обеспечение, порядок отбора победителей и призеров.
Основными целями Олимпиады являются: выявление и развитие интеллектуальных и творческих способностей и интереса к научной деятельности, творческой деятельности у талантливой молодежи; популяризация (пропаганда) научных знаний; создание условий для интеллектуального развития и поддержки одаренных школьников; оказание содействия молодежи в профессиональной ориентации и выборе образовательных траекторий.
Олимпиада проводится Пермским филиалом федерального государственного автономного образовательного учреждения высшего образования Национальный исследовательский университет Высшая школа экономики (далее НИУ ВШЭ  Пермь) в рамках организации и проведения профильного обучения и профессиональной ориентации обучающихся общеобразовательных организаций проекта Открытый университет, реализуемого совместно с Министерством образовании и науки Пермского края в соответствии с постановлением Правительства Пермского края от 31.03.2021  195-п.
Олимпиада проводится по следующим предметам: математика, обществознание, английский язык, информатика.
Последовательность этапов проведения Олимпиады, условия и порядок участия школьников в олимпиадных состязаниях регулируются Регламентом Олимпиады (далее Регламент).
Соответствие предметов профилям Олимпиады ежегодно устанавливается решением Организационного комитета Олимпиады, которое оформляется протоколом и размещается на странице Олимпиады в сети Интернет.
Для обеспечения единого информационного пространства для участников 
и организаторов Олимпиады создана страница Олимпиады на корпоративном сайте (портале) НИУ ВШЭ  Пермь в информационно-телекоммуникационной сети Интернет (далее сеть Интернет) по адресу:
:..2023.

Порядок организации и проведения Олимпиады
Для организационно-методического обеспечения Олимпиады создаются Организационный комитет (далее Оргкомитет) и методическая комиссия. Председателем Оргкомитета является заместитель директора НИУ ВШЭ  Пермь. Состав Оргкомитета утверждается приказом НИУ ВШЭ  Пермь. 
Оргкомитет и методическая комиссия Олимпиады формируются из профессорско-преподавательского состава и иных категорий работников
НИУ ВШЭ  Пермь. Состав методической комиссии утверждается ежегодно председателем Оргкомитета Олимпиады, протокол с решением председателя Оргкомитета размещается на странице НИУ ВШЭ  Пермь корпоративного сайта (портала) НИУ ВШЭ.
Оргкомитет Олимпиады:
устанавливает сроки проведения этапов Олимпиады;
разрабатывает расписание проведения олимпиадных состязаний;
обеспечивает непосредственное проведение Олимпиады;
аннулирует результаты участников в случае нарушения ими правил участия в Олимпиаде, установленных Регламентом;
формирует рейтинговые таблицы участников этапов Олимпиады в порядке, установленном Регламентом, и публикует на странице Олимпиады на странице НИУ ВШЭ  Пермь корпоративного сайта (портала) НИУ ВШЭ в сети Интернет;
совместно с методической комиссией Олимпиады определяет и утверждает списки победителей и призеров Олимпиады;
заблаговременно информирует совершеннолетних лиц, заявивших о своем участии в Олимпиаде, родителей (законных представителей) несовершеннолетних лиц, заявивших о своем участии в Олимпиаде;
обеспечивает сбор и хранение согласий совершеннолетних лиц, заявивших о своем участии в Олимпиаде, несовершеннолетних в возрасте от 14 до 18 лет, заявивших о своем участии в Конференции, также родителей (законных представителей) несовершеннолетних лиц, заявивших о своем участии в Олимпиаде на сбор, хранение, использование, распространение (предоставление) персональных данных;
осуществляет иные функции в соответствии с Положением и Регламентом.
Методическая комиссия Олимпиады:
разрабатывает задания для всех этапов Олимпиады;
разрабатывает критерии и методики оценки выполненных заданий всех этапов Олимпиады;
проверяет и оценивает результаты выполнения олимпиадных заданий;
аннулирует результаты участников в случае выявления при проверке и оценивании работ плагиата, списывания и других фактов, указывающих на несамостоятельное выполнение олимпиадного задания. Решение методической комиссии об аннулировании результатов участников в случае выявления при проверке и оценивании работ факта нарушения участниками правил участия в Олимпиаде оформляется протоколом, который подписывается не менее чем двумя членами методической комиссии и председателем методической комиссии Олимпиады;
определяет на совместном заседании с Оргкомитетом победителей и призеров Олимпиады;
представляет в Оргкомитет предложения по совершенствованию Олимпиады;
осуществляет иные функции в соответствии с Положением и Регламентом.
Олимпиада проводится в два этапа. Первый (отборочный) этап проводится в заочной форме, второй (заключительный) этап в очной форме.
Первый (отборочный) этап проводится в форме выполнения олимпиадных заданий в дистанционном формате в режиме о с использованием сети Интернет по всем предметам математике, английскому языку, информатике и обществознанию.
Второй (заключительный) этап Олимпиады проводится в очной форме.
При проведении Олимпиады предусмотрено две возрастные группы в соответствии с классом обучения школьников (далее возрастные группы): группа А обучающиеся 11 класса; группа Б обучающиеся 10 класса.
Рабочим языком проведения Олимпиады является русский язык.

Порядок участия в Олимпиаде и определения победителей и призеров
В Олимпиаде на добровольной основе принимают индивидуальное участие ученики 10-11 классов, обучающиеся по образовательным программам среднего общего образования, в том числе лица, осваивающие образовательные программы среднего общего образования в форме семейного образования или самообразования, а также лица, осваивающие указанные образовательные программы за рубежом.
Участники Олимпиады в обязательном порядке должны пройти процедуру регистрации в соответствии с правилами, установленными в Регламенте. Совершая действия по регистрации, участник Олимпиады подтверждает, что он ознакомился с Положением и Регламентом Олимпиады.
Участники Олимпиады обязаны при регистрации представить в электронном виде согласие на обработку персональных данных. 
Взимание платы за участие в Олимпиаде не допускается.
По каждому предмету (и соответствующему профилю) Олимпиады к участию во втором (заключительном) этапе допускаются:
участники, успешно прошедшие первый (отборочный) этап;
победители и призеры Олимпиады предшествующего учебного года в случае, если они продолжают обучение в организациях, осуществляющих образовательную деятельность по образовательным программам среднего общего образования, а также в форме семейного образования или самообразования, при условии представления согласий на обработку персональных данных на странице НИУ ВШЭ  Пермь корпоративного сайта (портала) НИУ ВШЭ в сети Интернет.
Победители и призеры Олимпиады предшествующего учебного года допускаются без выполнения заданий первого (отборочного) этапа к выполнению заданий второго (заключительного) этапа.
Победители и призеры этапов Олимпиады определяются путем оценивания зашифрованных (обезличенных) работ участников Олимпиады на основании рейтинговой таблицы участников Олимпиады, сформированной на основании суммы баллов, полученной участником за выполнение заданий в своей возрастной группе.
Количество участников, успешно прошедших первый (отборочный) этап Олимпиады, по каждому предмету не должно превышать 50 процентов от общего фактического числа участников первого (отборочного) этапа Олимпиады по данному предмету в каждой возрастной группе.
Победители и призеры Олимпиады определяются по результатам второго (заключительного) этапа.
Победителями становятся участники второго (заключительного) этапа, набравшие максимальное количество баллов в своей возрастной группе при прохождении этого этапа. Возможно определение нескольких победителей при равном количестве баллов.
Призерами становятся участники, набравшие количество баллов, соответствующее второй и третьей позиции в рейтинге участников второго (заключительного) этапа. Возможно определение нескольких победителей при равном количестве баллов.
Победителям Олимпиады вручаются дипломы победителей Олимпиады (диплом степени), призерам Олимпиады дипломы призеров олимпиады (дипломы и степени).
Победители и призеры Олимпиады 10 класса получают право в следующем учебном году быть допущенными к участию во втором (заключительном) этапе Олимпиады (минуя отборочный этап) по соответствующему предмету в случае, если они продолжают обучение в организациях, осуществляющих образовательную деятельность по образовательным программам среднего общего образования, а также в форме семейного образования или самообразования.
Порядок участия, сроки и места проведения Олимпиады, информация 
о победителях и призерах доводятся до сведения участников путем ее размещения
 на странице НИУ ВШЭ  Пермь корпоративного сайта (портала) НИУ ВШЭ в сети Интернет."""


tokenizer = AutoTokenizer.from_pretrained("../models/sota")


vocab_size = len(tokenizer)
student_model = BiLSTM(
    vocab_size=vocab_size,
    embed_dim=EMBED_DIM,
    enc_hidden_dim=ENC_HIDDEN_DIM,
    dec_hidden_dim=DEC_HIDDEN_DIM,
    pad_idx=tokenizer.pad_token_id,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
).to(DEVICE)

# ckpt = torch.load("students/v3_2/e50/student_model.pt", map_location=DEVICE)
ckpt = torch.load("../models/lstm/student_model.pt", map_location=DEVICE)
# ckpt = torch.load("students/d7/e5/student_model.pt", map_location=DEVICE)
student_model.load_state_dict(ckpt)
student_model.eval()

inputs = tokenizer(
    test_text, max_length=MAX_SOURCE_LEN, truncation=True, return_tensors="pt"
)
src_ids = inputs["input_ids"].to(DEVICE)

summary = beam_search_decode(
    model=student_model,
    src_ids=src_ids,
    tokenizer=tokenizer,
    beam_size=8,
    max_length=120,
    min_len=10,
    device=DEVICE,
)
print(f"Sample: {summary}")

/var/folders/8n/y27w9hnn0q5dqzq6m21wkz_w0000gn/T/ipykernel_56799/186285821.py:116: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("../models/lstm/student_mo

Sample: Положение регламентирует организацию и проведение конкурса по решению кейсов для абитуриентов магистратуры НИУ ВШЭ Нижний Новгород по направлению «Прикладная математика и информатика» и «Прикладная математика и информатика» и «Прикладная математика и информатика» НИУ ВШЭ Нижний Новгород, устанавливая цели, задачи, порядок проведения и критерии оценки, а также требования к подаче заявок на обработку персональных данных. Конкурс проходит в два этапа – дистанционный отборочный и заключ


In [4]:
total = sum(p.numel() for p in student_model.parameters() if p.requires_grad)
print(f"Total params: {total:,}")

Total params: 45,697,221
